# Exercise 3
## Lillian Mueller and Regina Hong 
Investigating linear regression for Iris Dataset

In [1]:
from sklearn import (
    linear_model, 
    preprocessing, 
    model_selection, 
    metrics, 
    tree)
from sklearn.datasets import load_iris
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# loading in the iris dataset from sklearn
iris_data = load_iris()

# turning dataset into dataframe format for easier reading
df_iris = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)

df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
# adding the target as a class column and renaming numbers to class names
# 0 = setosa, 1 = versicolor, 2 = virginica

df_iris['class'] = iris_data.target.tolist()
df_iris['class'].replace({0: 'setosa', 1: 'versicolor', 2: 'virginica'},
                inplace=True)

#Create a new column for the target (to be ready for processing)
label_obj = preprocessing.LabelEncoder()
df_iris['class_level'] = label_obj.fit_transform(df_iris['class'])

df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class,class_level
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0
2,4.7,3.2,1.3,0.2,setosa,0
3,4.6,3.1,1.5,0.2,setosa,0
4,5.0,3.6,1.4,0.2,setosa,0


In [10]:
# Creating Train and Test datasets
predictors, target = iris_data.data, iris_data.target
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    predictors, 
    target, 
    test_size = 0.33)

# create logistic regression to classify iris
logR = linear_model.LogisticRegression().fit(x_train, y_train)

# find coefficients 
B0 = logR.intercept_[0]
B1 = logR.coef_[0][0]
B2 = logR.coef_[0][1]
B3 = logR.coef_[0][2]
B4 = logR.coef_[0][3]
print('B0: ', B0)
print('B1: ', B1)
print('B2: ', B2)
print('B3: ', B3)
print('B4: ', B4)

#mu = -B0/B1
#s = 1/B1
#print('mu: ', mu)
#print('s: ', s)

B0:  9.158107949967366
B1:  -0.4334429480590375
B2:  0.8442238629054185
B3:  -2.2683088119150674
B4:  -0.9609511440474477


In [21]:
# Test model with testing dataset and find accuracy 
logR_pred = logR.predict(x_test)
accuracy = {
    'LogReg' : [
        metrics.accuracy_score(y_train, logR.predict(x_train)), 
        metrics.accuracy_score(y_test, logR_pred),
        metrics.r2_score(y_test, logR_pred)
    ]
}
print(accuracy['LogReg'])

[0.97, 0.98, 0.9721603563474388]


In [22]:
# create decision tree model using same training and test datasets 
dTree = tree.DecisionTreeClassifier(criterion='gini').fit(x_train, y_train)

# Test model with testing dataset and find accuracy 
dtree_pred = dTree.predict(x_test)
accuracy['DTree'] = [
        metrics.accuracy_score(y_train, dTree.predict(x_train)), 
        metrics.accuracy_score(y_test, dtree_pred),
        metrics.r2_score(y_test, dtree_pred)
    ]

print(accuracy['DTree'])

[1.0, 0.98, 0.9721603563474388]


In [23]:
# Compare the accuracy of each model 
pd.DataFrame.from_dict(
    accuracy,
    orient='index', 
    columns=[
        'Train Data Accuracy', 
        'Test Data Accuracy', 
        'r2 Score'
    ])

,Train Data Accuracy,Test Data Accuracy,r2 Score
LogReg,0.97,0.98,0.97216
DTree,1.00,0.98,0.97216


In [14]:
def run_dt(criterion, X_train, X_test, Y_train, Y_test):

    x_train, x_test, y_train, y_test = X_train, X_test, Y_train, Y_test

    dTree = tree.DecisionTreeClassifier(criterion=criterion).fit(x_train, y_train)
    dtree_pred = dTree.predict(x_test)
    
    return [ 
        metrics.accuracy_score(y_train, dTree.predict(x_train)), 
        metrics.accuracy_score(y_test, dtree_pred),
        metrics.r2_score(y_test, dtree_pred)
    ]

In [15]:
# set max_iter parameter to 1000 so that model woulod converge
def run_LR(penalty, X_train, X_test, Y_train, Y_test):

    x_train, x_test, y_train, y_test = X_train, X_test, Y_train, Y_test
    
    logR = linear_model.LogisticRegression(penalty=penalty,
                                          max_iter=1000).fit(x_train, y_train)
    
    # find coefficients 
    B0 = logR.intercept_[0]
    B1 = logR.coef_[0][0]    
    
    mu = -B0/B1
    s = 1/B1

    return [
        metrics.accuracy_score(y_train, logR.predict(x_train)), 
        metrics.accuracy_score(y_test, logR_pred),
        metrics.r2_score(y_test, logR_pred)
    ]

In [16]:
print('Run dt model with gini inpurity 1000 times')
dt_scores = []
for i in range(1000):
    dt_scores.append(run_dt('gini', x_train, x_test, y_train, y_test))

dt_score_df = pd.DataFrame(dt_scores, columns=['Train Data Accuracy', 'Test Data Accuracy', 'r2 Score'])
print(dt_score_df.describe().round(6))

Run dt model with gini inpurity 1000 times
       Train Data Accuracy  Test Data Accuracy    r2 Score
count               1000.0             1000.00  1000.00000
mean                   1.0                0.98     0.97216
std                    0.0                0.00     0.00000
min                    1.0                0.98     0.97216
25%                    1.0                0.98     0.97216
50%                    1.0                0.98     0.97216
75%                    1.0                0.98     0.97216
max                    1.0                0.98     0.97216


In [17]:
print('Run LR model with L2 penalty 1000 times')
LR_L2_scores = []
for i in range(1000):
    LR_L2_scores.append(run_LR('l2', x_train, x_test, y_train, y_test))

LR_L2_score_df = pd.DataFrame(LR_L2_scores, columns=['Train Data Accuracy', 'Test Data Accuracy', 'r2 Score'])
print(LR_L2_score_df.describe().round(6))

Run LR model with L2 penalty 1000 times
       Train Data Accuracy  Test Data Accuracy    r2 Score
count              1000.00             1000.00  1000.00000
mean                  0.97                0.98     0.97216
std                   0.00                0.00     0.00000
min                   0.97                0.98     0.97216
25%                   0.97                0.98     0.97216
50%                   0.97                0.98     0.97216
75%                   0.97                0.98     0.97216
max                   0.97                0.98     0.97216


In [18]:
print('Run LR model with No penalty 1000 times')
LR_N_scores = []
for i in range(1000):
    LR_N_scores.append(run_LR(None, x_train, x_test, y_train, y_test))

LR_N_score_df = pd.DataFrame(LR_N_scores, columns=['Train Data Accuracy', 'Test Data Accuracy', 'r2 Score'])
print(LR_N_score_df.describe().round(6))

Run LR model with No penalty 1000 times
       Train Data Accuracy  Test Data Accuracy    r2 Score
count               1000.0             1000.00  1000.00000
mean                   1.0                0.98     0.97216
std                    0.0                0.00     0.00000
min                    1.0                0.98     0.97216
25%                    1.0                0.98     0.97216
50%                    1.0                0.98     0.97216
75%                    1.0                0.98     0.97216
max                    1.0                0.98     0.97216


In [19]:
# create ranking of results of logistic regression model 
probRanks = pd.DataFrame(
    logR.predict_proba(predictors), 
    columns=[
        'Prob of setosa (0)', 
        'Prob of versicolor (1)', 
        'Prob of virginica (2)'
    ]
)
probRanks.sample(10).round(3)

,Prob of setosa (0),Prob of versicolor (1),Prob of virginica (2)
103,0.000,0.112,0.888
2,0.979,0.021,0.000
27,0.966,0.034,0.000
86,0.003,0.777,0.219
138,0.003,0.464,0.534
141,0.000,0.155,0.845
66,0.012,0.757,0.231
112,0.000,0.092,0.908
14,0.982,0.018,0.000
99,0.025,0.916,0.060


In [26]:
pd.set_option('display.max_rows', None)
probRanks.round(3)

,Prob of setosa (0),Prob of versicolor (1),Prob of virginica (2)
0,0.974,0.026,0.000
1,0.962,0.038,0.000
2,0.979,0.021,0.000
3,0.968,0.032,0.000
4,0.979,0.021,0.000
5,0.959,0.041,0.000
6,0.981,0.019,0.000
7,0.968,0.032,0.000
8,0.973,0.027,0.000
9,0.959,0.041,0.000


In [28]:
df_iris.sort_values(by=['sepal length (cm)'])

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class,class_level
13,4.3,3.0,1.1,0.1,setosa,0
42,4.4,3.2,1.3,0.2,setosa,0
38,4.4,3.0,1.3,0.2,setosa,0
8,4.4,2.9,1.4,0.2,setosa,0
41,4.5,2.3,1.3,0.3,setosa,0
22,4.6,3.6,1.0,0.2,setosa,0
3,4.6,3.1,1.5,0.2,setosa,0
6,4.6,3.4,1.4,0.3,setosa,0
47,4.6,3.2,1.4,0.2,setosa,0
2,4.7,3.2,1.3,0.2,setosa,0


In [20]:
# rank two new records 
records = [
    [5.8,2.8,5.1,2.4],
    [6.0,2.2,4.0,1.0]
]
newRecords = pd.DataFrame(
    records, 
    columns=[
        'sepal length (cm)',
        'sepal width (cm)',
        'petal length (cm)',
        'petal width (cm)'
    ]
)

# making prediction with model 
newRecords['Prediction'] = logR.predict(records)

# adding rankings to dataframe
rankProbs = pd.DataFrame(
    logR.predict_proba(records),
    columns=[
        'Prob of setosa (0)', 
        'Prob of versicolor (1)', 
        'Prob of virginica (2)'
        ]
    )
newRecords[rankProbs.columns] = rankProbs

newRecords

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Prediction,Prob of setosa (0),Prob of versicolor (1),Prob of virginica (2)
0,5.8,2.8,5.1,2.4,2,0.000172,0.069843,0.929985
1,6.0,2.2,4.0,1.0,1,0.012885,0.964954,0.022162
